# Pupil diameter
- Preprocess noisy pupil diameter signal
- Apply subtractive baseline correction

In [ ]:
# Required libraries
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Load the csv file containing eye tracker data into a pandas DataFrame
data = pd.read_csv("sample_eye_tracking_data.csv")

# Display the first 5 rows (from index 10_000 onwards, for a display sample)
data[10_000:].head()

## Visualizing the pupil diameter signal
Visually inspecting the pupil diameter signal to check for noise and artifacts is an important part of pupillometry preprocessing ([Mathôt et al., 2018]).

[Mathôt et al., 2018]: https://doi.org/10.3758/s13428-017-1007-2

Since we will be plotting the pupil diameter many times for visual inspection, and also to check if the preprocessing pipelines have managed to successfully clean the data, it makes sense to create a function for this visualization.

Below, we will create a simple function that encapsulates the plotting script, for ease of use within this notebook.

In [ ]:
def plot_pupil_dia(df):
    """
    Plots the diameter and validity of the left pupil throughout the recording session.

    Args:
        - df (pd.DataFrame): The dataframe containing eye-tracking data.
    """
    
    plt.figure(figsize=(15, 5))
    plt.plot(df.left_pupil_diameter, label='Left pupil diameter', color='k') # plot the left pupil diameter signal
    plt.plot(df.left_pupil_validity, label='Left pupil validity') # plot the validty (either 0 or 1) of left pupil
    plt.title('Left pupil diameter throughout the experiment session')
    plt.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
# Usage
plot_pupil_dia(data)

A study-specific visualization that is able to convey the various segments of a recording session can also be helpful.

My study-specific viz function, `overview_plot` can be found in `experiment_specific_utils.common`. <br>
This function applies a color to the different segments of my recording: fixation cross segment, scrambled image segment, and stimulus image segment.

In [ ]:
from experiment_specific_utils.common import overview_plot
help(overview_plot)

overview_plot(data)

# Preprocessing the pupil diameter signal

## Removal of invalid data
Use the function `remove_invalids` from `compute_eye_metrics.utils` to drop all rows that do not contain pupil diameter data (likely due to blinks), and noisy data that surrounds a blink.

In [ ]:
# Import the function and print docstring
from compute_eye_metrics.utils import remove_invalids
help(remove_invalids)

In [ ]:
# Usage
data_invalids_removed = remove_invalids(data)

In [ ]:
plot_pupil_dia(data_invalids_removed)

## Replacing invalid data
Use the function `replace_invalids_w_nans` from `compute_eye_metrics.utils` to replace all rows that do not contain pupil diameter. Invalid data will be replaced with `np.nan`.

This function ensures that the length of the data stays the same, while ensuring the removal of noisy data.

*Note: While not used for pupil diameter, this function is a crucial component of saccade extraction.*

In [ ]:
# Import the function and print docstring
from compute_eye_metrics.utils import replace_invalids_w_nans
help(replace_invalids_w_nans)

In [ ]:
# Usage
data_invalids_replaced = replace_invalids_w_nans(data)

In [ ]:
plot_pupil_dia(data_invalids_replaced)

## Applying smoothing to the pupil diameter signal
Use the function `apply_smoothing` from `compute_eye_metrics.pupildiameter` to smooth the pupil diameter signal.

In [ ]:
# Import the function and print docstring
from compute_eye_metrics.pupildiameter import apply_smoothing
help(apply_smoothing)

In [ ]:
# Usage
# Note: Use the dataframe from which invalid and noisy data has been removed
data_invalids_removed_smoothed = apply_smoothing(data_invalids_removed, plot_fig=True)

# Performing baseline correction of pupil diameter
Use the function `baseline_correction` from `compute_eye_metrics.pupildiameter` to perform **subtractive baseline correction** of the pupil diameter signal.

The signal is corrected by subtracting the baseline (scrambled image) value from its corresponding stimulus period. <br>
The baseline value is calculated by taking the mean of a 500 ms period, towards the end of the scrambled image viewing period.

Each pupil is baseline corrected individually.

The dataframe returned by this function has two new columns, containing baseline-corrected pupil diameter values: <br>
`left_pupil_diameter_bc` and `left_pupil_diameter_bc`.

In [ ]:
# Import the function and print docstring
from compute_eye_metrics.pupildiameter import baseline_correction
help(baseline_correction)

In [ ]:
# Usage
# Note: Use the dataframe where invalids have been removed, and smoothing applied
data_baseline_corrected = baseline_correction(data_invalids_removed_smoothed)

In [ ]:
# Visualize to check
# (this time, plot the new columns obtained after running baseline correction: left|right_pupil_diameter_bc)
plt.figure(figsize=(15, 5))
plt.plot(data_baseline_corrected.left_pupil_diameter_bc, label='Left pupil diameter (baseline-corrected)', color='k')
plt.title('Left pupil diameter throughout the experiment session')
plt.legend()
plt.tight_layout()
plt.show()

In the plot above, the missing segments are those which were used as baseline (scrambled image). In this implementation, the baseline segments (where `df.remarks=="SCRAMBLED IMAGE"`) do not hold any baseline corrected pupil diameter value.

# Getting the pupil diameter
After preprocessing and baseline-correcting the pupil diameter signal, the mean and sd of the pupil diameter can be computed. This is study-specific, depending on which exact segment/task the metrics need to be computed for.

Shown below is a very general overview of the mean and sd for both absolute and baseline-corrected pupil diameters, computed for the entire recording session.

In [ ]:
print("PUPIL DIAMETER (ABSOLUTE, AFTER SMOOTHING) - MM")
print("Left mean:\t", data_baseline_corrected.left_pupil_diameter_smooth.mean())
print("Left std:\t", data_baseline_corrected.left_pupil_diameter_smooth.std())
print("Right mean:\t", data_baseline_corrected.right_pupil_diameter_smooth.mean())
print("Right std:\t", data_baseline_corrected.right_pupil_diameter_smooth.std())

print("\nPUPIL DIAMETER (BASELINE-CORRECTED) - MM")
print("Left mean:\t", data_baseline_corrected.left_pupil_diameter_bc.mean())
print("Left std:\t", data_baseline_corrected.left_pupil_diameter_bc.std())
print("Right mean:\t", data_baseline_corrected.right_pupil_diameter_bc.mean())
print("Right std:\t", data_baseline_corrected.right_pupil_diameter_bc.std())